In [1]:
import pandas as pd
import sys
import os
sys.path.append("../")
sys.path.append("../..")
sys.path.append("../../..")

from clust.transformation.splitDataByCycle.cycleModule import CycleData
from clust.transformation.splitDataByCycle import dataByCycle
from clust.preprocessing.dataPreprocessing import DataPreprocessing

## InfluxDB client

In [2]:
from clust.ingestion.influx import influx_Client_v2 as influx_Client, multipleDataSets
from setting import influx_setting_KETI as ins

DBClient = influx_Client.influxClient(ins.CLUSTDataServer2)

# Data Cycle Split Example

In [3]:
# DB Setting
db_name ='air_indoor_도서관'
ms_name ='ICW0W2000087'
"""
db_name ='farm_outdoor_weather_clean'
ms_name ='gunwi'
"""
days = 50000
############
last_time = DBClient.get_last_time(db_name, ms_name)
rawData = DBClient.get_data_by_days(last_time, str(days), db_name, ms_name)

In [4]:
# Check Data
rawData

,in_co2,in_humi,in_noise,in_pm01,in_pm10,in_pm25,in_temp,in_voc
time,,,,,,,,
2020-05-14 16:33:00+00:00,693.0,19.0,59.0,6.0,21.0,10.0,29.7,19.0
2020-05-14 16:34:00+00:00,604.0,20.0,58.0,6.0,23.0,10.0,29.6,35.0
2020-05-14 16:35:00+00:00,741.0,20.0,59.0,5.0,24.0,10.0,29.6,68.0
2020-05-14 16:36:00+00:00,844.0,20.0,59.0,5.0,24.0,10.0,29.6,77.0
2020-05-14 16:37:00+00:00,805.0,19.0,61.0,6.0,23.0,10.0,29.6,65.0
...,...,...,...,...,...,...,...,...
2022-04-30 23:55:00+00:00,212.0,37.0,45.0,19.0,24.0,12.0,19.1,12.0
2022-04-30 23:56:00+00:00,212.0,37.0,46.0,20.0,24.0,12.0,19.1,15.0
2022-04-30 23:57:00+00:00,209.0,37.0,45.0,20.0,24.0,12.0,19.1,15.0


In [5]:
# Check last_time, 
print(last_time)

2022-04-30 23:59:00+00:00


In [6]:
# Check Frequency
freq = DBClient.get_freq(db_name, ms_name)
print(freq)
print(rawData.index[0], rawData.index[1])

0 days 00:01:00
2020-05-14 16:33:00+00:00 2020-05-14 16:34:00+00:00


In [7]:
# fill in the missing time index
refine_param = {'removeDuplication': {'flag': True} , 'staticFrequency': {'flag': True, 'frequency': None}}
output = DataPreprocessing().get_refinedData(rawData, refine_param)

# Function

True = Entire Cycle Data

False = Prefect Cycle Data

ex) .getHourCyccleSet(data, num, FullCycle)

data = dataframe

num = hour * (count)

## Hour Cycle

In [8]:
# Get Split Cycle Data By Hour
hourCycle = CycleData().getHourCycleSet(output,3, False)
hourCycle

[                           in_co2  in_humi  in_noise  in_pm01  in_pm10  \
 time                                                                     
 2020-05-14 17:00:00+00:00   508.0     19.0      58.0      8.0     26.0   
 2020-05-14 17:01:00+00:00   517.0     19.0      60.0      7.0     25.0   
 2020-05-14 17:02:00+00:00   513.0     19.0      59.0      6.0     25.0   
 2020-05-14 17:03:00+00:00   502.0     19.0      60.0      6.0     24.0   
 2020-05-14 17:04:00+00:00   501.0     19.0      59.0      6.0     25.0   
 ...                           ...      ...       ...      ...      ...   
 2020-05-14 19:55:00+00:00   513.0     22.0      53.0     10.0     26.0   
 2020-05-14 19:56:00+00:00   512.0     22.0      57.0      9.0     28.0   
 2020-05-14 19:57:00+00:00   519.0     22.0      55.0      9.0     27.0   
 2020-05-14 19:58:00+00:00   509.0     22.0      54.0      9.0     26.0   
 2020-05-14 19:59:00+00:00   515.0     22.0      54.0     10.0     25.0   
 
                       

## Day Cycle

In [9]:
# Get Split Cycle Data By Day
dayCycle = CycleData().getDayCycleSet(output,3,False)
dayCycle

[                           in_co2  in_humi  in_noise  in_pm01  in_pm10  \
 time                                                                     
 2020-05-15 00:00:00+00:00   521.0     25.0      52.0      8.0     25.0   
 2020-05-15 00:01:00+00:00   518.0     25.0      51.0      8.0     23.0   
 2020-05-15 00:02:00+00:00   517.0     25.0      52.0      8.0     24.0   
 2020-05-15 00:03:00+00:00   516.0     25.0      53.0      8.0     25.0   
 2020-05-15 00:04:00+00:00   517.0     25.0      56.0      8.0     23.0   
 ...                           ...      ...       ...      ...      ...   
 2020-05-17 23:55:00+00:00     NaN      NaN       NaN      NaN      NaN   
 2020-05-17 23:56:00+00:00     NaN      NaN       NaN      NaN      NaN   
 2020-05-17 23:57:00+00:00     NaN      NaN       NaN      NaN      NaN   
 2020-05-17 23:58:00+00:00     NaN      NaN       NaN      NaN      NaN   
 2020-05-17 23:59:00+00:00     NaN      NaN       NaN      NaN      NaN   
 
                       

## Week Cycle

In [10]:
# Get Split Cycle Data By Week
weekCycle = CycleData().getWeekCycleSet(output, 3, False)
weekCycle

[                           in_co2  in_humi  in_noise  in_pm01  in_pm10  \
 time                                                                     
 2020-05-18 00:00:00+00:00     NaN      NaN       NaN      NaN      NaN   
 2020-05-18 00:01:00+00:00     NaN      NaN       NaN      NaN      NaN   
 2020-05-18 00:02:00+00:00     NaN      NaN       NaN      NaN      NaN   
 2020-05-18 00:03:00+00:00     NaN      NaN       NaN      NaN      NaN   
 2020-05-18 00:04:00+00:00     NaN      NaN       NaN      NaN      NaN   
 ...                           ...      ...       ...      ...      ...   
 2020-06-07 23:55:00+00:00   447.0     48.0      47.0     18.0     23.0   
 2020-06-07 23:56:00+00:00   440.0     48.0      46.0     18.0     23.0   
 2020-06-07 23:57:00+00:00   443.0     48.0      47.0     18.0     24.0   
 2020-06-07 23:58:00+00:00   442.0     48.0      47.0     18.0     25.0   
 2020-06-07 23:59:00+00:00   443.0     49.0      47.0     18.0     23.0   
 
                       

## Month Cycle

In [11]:
# Get Split Cycle Data By Month
monthCycle = CycleData().getMonthCycleSet(output, 3, False)
monthCycle

[                           in_co2  in_humi  in_noise  in_pm01  in_pm10  \
 time                                                                     
 2020-06-01 00:00:00+00:00   448.0     45.0      47.0     16.0     17.0   
 2020-06-01 00:01:00+00:00   450.0     45.0      47.0     17.0     17.0   
 2020-06-01 00:02:00+00:00   451.0     45.0      47.0     16.0     16.0   
 2020-06-01 00:03:00+00:00   450.0     45.0      47.0     17.0     17.0   
 2020-06-01 00:04:00+00:00   458.0     45.0      47.0     16.0     17.0   
 ...                           ...      ...       ...      ...      ...   
 2020-08-31 23:55:00+00:00   375.0     58.0      46.0      0.0      0.0   
 2020-08-31 23:56:00+00:00   373.0     58.0      46.0      0.0      0.0   
 2020-08-31 23:57:00+00:00   375.0     58.0      46.0      0.0      0.0   
 2020-08-31 23:58:00+00:00   372.0     58.0      46.0      0.0      0.0   
 2020-08-31 23:59:00+00:00   377.0     58.0      46.0      0.0      0.0   
 
                       

## Year Cycle

In [12]:
# 아직 작업중
# Get Split Cycle Data By Year
yearCycle = CycleData().getYearCycleSet(output, 1, False)
yearCycle

[                           in_co2  in_humi  in_noise  in_pm01  in_pm10  \
 time                                                                     
 2021-01-01 00:00:00+00:00   393.0     19.0      46.0     13.0     19.0   
 2021-01-01 00:01:00+00:00   393.0     19.0      46.0     13.0     21.0   
 2021-01-01 00:02:00+00:00   397.0     20.0      46.0     14.0     23.0   
 2021-01-01 00:03:00+00:00   390.0     19.0      46.0     13.0     22.0   
 2021-01-01 00:04:00+00:00   392.0     20.0      46.0     13.0     22.0   
 ...                           ...      ...       ...      ...      ...   
 2021-12-31 23:55:00+00:00   226.0     20.0      46.0      6.0     10.0   
 2021-12-31 23:56:00+00:00   221.0     20.0      46.0      6.0     10.0   
 2021-12-31 23:57:00+00:00   220.0     20.0      46.0      6.0     10.0   
 2021-12-31 23:58:00+00:00   227.0     20.0      46.0      6.0      9.0   
 2021-12-31 23:59:00+00:00   223.0     20.0      46.0      6.0      9.0   
 
                       

# Cycle DataSet Example

## DataSet

In [13]:
# DB, MS, Time Setting
db_name="air_indoor_도서관"
ms_name="ICW0W2000087"
start_time = '2020-05-30T00:00:00Z'
end_time = '2020-06-20T23:00:00Z'

## Cycle Parameter

In [14]:
# Cycle Parameter Setting - cycle, cycle_time
feature_cycle = 'Day'
feature_cycle_times = 1

## Cycle Data

In [15]:
# Get data of the specific measurement based on start-end duration
data = DBClient.get_data_by_time(start_time, end_time, db_name, ms_name)
data

,in_co2,in_humi,in_noise,in_pm01,in_pm10,in_pm25,in_temp,in_voc
time,,,,,,,,
2020-05-30 00:00:00+00:00,496.0,46.0,46.0,14.0,18.0,9.0,25.2,28.0
2020-05-30 00:01:00+00:00,498.0,46.0,47.0,14.0,18.0,9.0,25.2,28.0
2020-05-30 00:02:00+00:00,491.0,46.0,47.0,14.0,17.0,8.0,25.2,28.0
2020-05-30 00:03:00+00:00,491.0,46.0,47.0,15.0,19.0,9.0,25.2,27.0
2020-05-30 00:04:00+00:00,495.0,46.0,47.0,15.0,18.0,9.0,25.2,28.0
...,...,...,...,...,...,...,...,...
2020-06-20 22:55:00+00:00,409.0,44.0,47.0,13.0,15.0,7.0,30.5,8.0
2020-06-20 22:56:00+00:00,410.0,44.0,46.0,13.0,14.0,7.0,30.5,9.0
2020-06-20 22:57:00+00:00,403.0,44.0,46.0,13.0,16.0,7.0,30.5,8.0


In [16]:
# Fill in the missing time index
refine_param = {'removeDuplication': {'flag': True}, 'staticFrequency': {'flag': True, 'frequency': None}}
data = DataPreprocessing().get_refinedData(data, refine_param)

In [17]:
# Get Cycle DataSet
cycle_data = dataByCycle.getCycleSelectDataSet(data, feature_cycle, feature_cycle_times)
cycle_data

{'2020-05-30 00:00:00':                            in_co2  in_humi  in_noise  in_pm01  in_pm10  \
 time                                                                     
 2020-05-30 00:00:00+00:00   496.0     46.0      46.0     14.0     18.0   
 2020-05-30 00:01:00+00:00   498.0     46.0      47.0     14.0     18.0   
 2020-05-30 00:02:00+00:00   491.0     46.0      47.0     14.0     17.0   
 2020-05-30 00:03:00+00:00   491.0     46.0      47.0     15.0     19.0   
 2020-05-30 00:04:00+00:00   495.0     46.0      47.0     15.0     18.0   
 ...                           ...      ...       ...      ...      ...   
 2020-05-30 23:55:00+00:00   472.0     43.0      47.0     17.0     19.0   
 2020-05-30 23:56:00+00:00   474.0     43.0      47.0     17.0     19.0   
 2020-05-30 23:57:00+00:00   474.0     43.0      47.0     17.0     18.0   
 2020-05-30 23:58:00+00:00   479.0     43.0      47.0     17.0     20.0   
 2020-05-30 23:59:00+00:00   473.0     43.0      48.0     17.0     18.0   
 
